# NEWS 파일(json)을 Bigquery에 저장

In [2]:
import os
import sys
import json
import requests
import pickle
import gzip
import pandas as pd
import pandas_gbq as gbq
from pathlib import Path
from google.oauth2 import service_account
from google.cloud import bigquery
from IPython.core.debugger import set_trace
from tqdm.auto import tqdm
tqdm.pandas()

## Configuration

In [3]:
proj = 'global-news-crawl'
table_downloaded = 'news_dataset.downloaded'
table_trashed = 'news_dataset.trashed'
credentials = service_account.Credentials.from_service_account_file('global-news-crawl-c48d7cd9aa81.json')

## Bigquery에서 기존 뉴스 ID 리스트 받기

In [22]:
%%time
qry_0 = 'SELECT id FROM `{}`'.format(proj + '.' + table_downloaded)
ids_downloaded = gbq.read_gbq(qry_0, project_id=proj, credentials=credentials)#, use_bqstorage_api=True)

Wall time: 18.2 s


In [23]:
%%time
qry_1 = 'SELECT id FROM `{}`'.format(proj + '.' + table_trashed)
ids_trashed = gbq.read_gbq(qry_1, project_id=proj, credentials=credentials)

Wall time: 12.7 s


In [24]:
ids_downloaded_set = set(ids_downloaded.id)
ids_trashed_set = set(ids_trashed.id)

In [25]:
assert len(ids_downloaded) == len(ids_downloaded_set), 'duplicated in downloaded'
assert len(ids_trashed) == len(ids_trashed_set), 'duplicated in downloaded'
assert len(ids_downloaded_set & ids_trashed_set) == 0, 'ids overlapping'

In [26]:
newsids = ids_downloaded_set | ids_trashed_set; len(newsids)

392315

In [27]:
len(ids_downloaded_set), len(ids_trashed_set)

(241521, 150794)

## 파일에서 뉴스정보 추출
나중에는 필요없는 과정이다. 다운로드 받은 즉시 바로 Bigquery에 전송할 것이므로

In [28]:
def extract_contents(newsids=None, where=None, n=1000):
    df = {}
    _n = 0
    
    for file in Path(where).glob('**/*.json'):
        id = file.stem
        
        if id not in newsids:
            try:
                js = json.loads(file.read_text())
            
                if 'authors' in js:
                    js['authors'] = ', '.join(js['authors'])

                df[id] = js

                _n += 1
                print('\r{}'.format(_n), end='')
                if _n == n: break
                    
            except:
                print(file)
           
    df = pd.DataFrame.from_dict(df, orient='index')
    df.index.name = 'id'
    return df.reset_index()

In [ ]:
df_downloaded = extract_contents(newsids=newsids, where='newsdata/downloaded', n=50000); print('\n')
df_trashed = extract_contents(newsids=newsids, where='newsdata/trashed', n=30000)

5

## downloaded와 trashed 간에 겹치는 게 없는지 확인
기존의 뉴스파일을 Bigquery에 전송하는 과정에서, 이 둘간에 겹치는 사례가 종종 있었다
(2019.10.18)

In [20]:
intersect = set(df_trashed.id) & set(df_downloaded.id)
assert len(intersect) == 0, 'ids overlapping'

## Bigquery에 업로드

In [21]:
gbq.to_gbq(df_downloaded, table_downloaded, project_id=proj, if_exists='append', chunksize=1000, credentials=credentials)
gbq.to_gbq(df_trashed, table_trashed, project_id=proj, if_exists='append', chunksize=1000, credentials=credentials)

3it [00:22,  7.42s/it]
2it [00:09,  4.60s/it]


# 아래 코드를 실험해봐야한다. __init__ 까지는 확인됨

In [28]:
proj = 'global-news-crawl'
table_downloaded = 'news_dataset.downloaded'
table_trashed = 'news_dataset.trashed'
credentials = service_account.Credentials.from_service_account_file('global-news-crawl-c48d7cd9aa81.json')

localpath_to_downloaded = 'newsdata/downloaded'
localpath_to_trashed = 'newsdata/trashed'


class Recorder:
    def __init__(self, storage='local'):
        self.storage = storage
        self.ids = self._get_ids(storage)

    def _query_ids_from_bigquery(self, tb):
        qry = 'SELECT id FROM `{}`'.format(proj + '.' + tb)
        return gbq.read_gbq(qry, credentials=credentials).id
        
    def _retreive_ids_from_local(self, path):
        return [p.stem for p in Path(path).glob('**/*.json')]
        
        
    def _get_ids(self, storage):
        print('checking ' + storage + ' storage... ', end='')

        if storage == 'bigquery':
            ids_downloaded = self._query_ids_from_bigquery(table_downloaded)
            ids_trashed = self._query_ids_from_bigquery(table_trashed)

        elif storage == 'local':
            ids_downloaded = self._retreive_ids_from_local(localpath_to_downloaded)
            ids_trashed = self._retreive_ids_from_local(localpath_to_trashed)

        ids_downloaded_set = set(ids_downloaded)
        ids_trashed_set = set(ids_trashed)
        
        if len(ids_downloaded) != len(ids_downloaded_set):
            '''downloaded articles의 uniqueness'''
            raise self.DuplicatesInSingleTable('duplicated in downloaded')

        if len(ids_trashed) != len(ids_trashed_set):
            '''trashed articles의 uniqueness'''
            raise self.DuplicatesInSingleTable('duplicated in trashed')

        if len(ids_downloaded_set & ids_trashed_set) != 0:
            '''downloaded와 trashed 간의 uniqueness'''
            raise self.DuplicatesBetweenTwoTables('duplicated between downloaded and trashed')
    
        ids = ids_downloaded_set | ids_trashed_set
        
        print('done')
        print('we have total {} ({} downloaded, {} trashed)'.format(len(ids), len(ids_downloaded_set), len(ids_trashed_set)))
        return ids


    def has(self, id):
        return id in self.ids


    def update(self, downloaded=None, trashed=None, chunksize=1000, subdir_len=3):
        if self.storage == 'bigquery':
            self._update_bigquery('downloaded', downloaded, chunksize)
            self._update_bigquery('trashed', trashed, chunksize)
        
        elif self.storage == 'local':
            self._update_local('downloaded', downloaded, subdir_len)
            self._update_local('trashed', trashed, subdir_len)
            

    def _update_local(self, newstype, newsdict, subdir_len):
        if newsdict is not None:
            if newstype == 'downloaded':
                path = localpath_to_downloaded
            elif newstype == 'trashed':
                path = localpath_to_trashed
            
            for id, article in newsdict.items():
                _dir = Path(path + '/' + id[:subdir_len])
                _dir.mkdir(parent=True, exist_ok=True)
                fname = id + '.json'
                fpath = _dir / fname
                with fpath.open('w') as f:
                    json.dump(article)


    def _update_bigquery(self, newstype, newsdict, chunksize):
        if newsdict is not None:
            if newstype == 'downloaded':
                tb = table_downloaded
            elif newstype == 'trashed':
                tb = table_trashed

            df = pd.DataFrame.from_dict(newsdict, orient='index')
            df.index.name = 'id'
            gbq.to_gbq(df.reset_index(), tb, project_id=proj, if_exists='append', chunksize=chunksize, credentials=credentials)
            

    class DuplicatesInSingleTable(Exception):
        pass

    class DuplicatesBetweenTwoTables(Exception):
        pass

In [31]:
recorder = Recorder(storage='bigquery')

checking bigquery storage... done
we have: total 385957 (237649 downloaded, 148308 trashed)


In [37]:
for src in tqdm(Path('newsdata/downloaded').glob('**/*.json')):
    p_dest = Path('newsdata/downloaded2/' + src.name[:3])
    p_dest.mkdir(parents=True, exist_ok=True)
    dest = p_dest / src.name
    dest.write_text(src.read_text())

In [41]:
len(list(Path('newsdata/downloaded').glob('**/*.json')))

238232

In [42]:
len(list(Path('newsdata/downloaded').glob('**/*.json')))

238608

In [43]:
238232+376

238608

In [70]:
testdict = {
    '111': {'a':1, 'b':2}, 
    '222': {'a':3, 'c':4}
}

In [73]:
testdf = pd.DataFrame.from_dict(testdict, orient='index')
testdf.index.name = 'idx'
testdf.reset_index()

,idx,a,b,c
0,111,1,2.0,NaN
1,222,3,NaN,4.0
